# Using Experiments with the MNIST Dataset and Keras

## Environment Setup

- Image: TensorFlow 2.6 CPU Optimized
- Kernel: Python 3
- Instance type: ml.t3.medium

## Background

This notebook uses the SageMaker SDK to demonstrate how to use experiments.  Using the MNIST dataset (a large database of handwritten digits), we use Keras to train the model.  To write log metrics back to Experiments, we use a Keras callback.

This notebook has been adapted from the [SageMaker examples](https://github.com/aws/amazon-sagemaker-examples/blob/main//sagemaker-experiments/local_experiment_tracking/keras_experiment.ipynb).

## Initialize Environment and Variables

In [ ]:
import sys

In [ ]:
# Make sure we have the latest versions of the SDKs
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade boto3
!{sys.executable} -m pip install --upgrade sagemaker
!{sys.executable} -m pip install --upgrade tensorflow

In [ ]:
import json
import boto3
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

# Get the SageMaker and boto sessions, plus the execution role from the SageMaker domain
sagemaker_session = Session()
boto_sess = boto3.Session()

role = get_execution_role()

default_bucket = sagemaker_session.default_bucket()

sm = boto_sess.client("sagemaker")
region = boto_sess.region_name

---
## Data

For this lesson, we're using the MNIST dataset, downloading it from the SageMaker sample files.

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [ ]:
!mkdir -p datasets

In [ ]:
num_classes = 10
input_shape = (28, 28, 1)

s3 = boto3.client("s3")

train_path = "datasets/input_train.npy"
test_path = "datasets/input_test.npy"
train_labels_path = "datasets/input_train_labels.npy"
test_labels_path = "datasets/input_test_labels.npy"

# Load the data and split it between train and test sets
s3.download_file("sagemaker-sample-files", "datasets/image/MNIST/numpy/input_train.npy", train_path)
s3.download_file("sagemaker-sample-files", "datasets/image/MNIST/numpy/input_test.npy", test_path)
s3.download_file(
    "sagemaker-sample-files", "datasets/image/MNIST/numpy/input_train_labels.npy", train_labels_path
)
s3.download_file(
    "sagemaker-sample-files", "datasets/image/MNIST/numpy/input_test_labels.npy", test_labels_path
)

In [ ]:
# Prepare the data for training
x_train = np.load(train_path)
x_test = np.load(test_path)
y_train = np.load(train_labels_path)
y_test = np.load(test_labels_path)

# Reshape the arrays
x_train = np.reshape(x_train, (60000, 28, 28))
x_test = np.reshape(x_test, (10000, 28, 28))
y_train = np.reshape(y_train, (60000,))
y_test = np.reshape(y_test, (10000,))

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Build the Model

In [ ]:
def get_model(dropout=0.5):
    """ """
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(dropout),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

## Define the Keras Callback

The Keras Callback class provides a method *on_epoch_end*, which emits metrics at the end of each epoch. All emitted metrics will be logged in the run passed to the callback.

In [ ]:
class ExperimentCallback(keras.callbacks.Callback):
    """ """

    def __init__(self, run, model, x_test, y_test):
        """Save params in constructor"""
        self.run = run
        self.model = model
        self.x_test = x_test
        self.y_test = y_test

    def on_epoch_end(self, epoch, logs=None):
        """ """
        keys = list(logs.keys())
        for key in keys:
            self.run.log_metric(name=key, value=logs[key], step=epoch)
            print("{} -> {}".format(key, logs[key]))

## Set up a SageMaker Experiment and its Runs, then Train

Next, we train the Keras model locally on the same instance where this notebook is running.  With each run, we track the input artifacts and write them to files.  We use the ExperimentCallback method to log the metrics to the Experiment run.

In [ ]:
from sagemaker.experiments.run import Run

batch_size = 20
epochs = 3
dropout = 0.5

model = get_model(dropout)

experiment_name = "mnist-keras-experiment"
run_name = "mnist-keras-batch-size-20"
with Run(experiment_name=experiment_name, run_name=run_name, sagemaker_session=sagemaker_session) as run:
    run.log_parameter("batch_size", batch_size)
    run.log_parameter("epochs", epochs)
    run.log_parameter("dropout", dropout)

    run.log_file("datasets/input_train.npy", is_output=False)
    run.log_file("datasets/input_test.npy", is_output=False)
    run.log_file("datasets/input_train_labels.npy", is_output=False)
    run.log_file("datasets/input_test_labels.npy", is_output=False)

    # Train locally
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.1,
        callbacks=[ExperimentCallback(run, model, x_test, y_test)],
    )

    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

    run.log_metric(name="Final Test Loss", value=score[0])
    run.log_metric(name="Final Test Accuracy", value=score[1])

In [ ]:
# Delete the experiment
from sagemaker.experiments.experiment import _Experiment

exp = _Experiment.load(experiment_name=experiment_name, sagemaker_session=sagemaker_session)
exp._delete_all(action="--force")